#Modeling on the SelfMA Dataset

First, some EDA.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

self_MA = pd.read_json('/content/drive/MyDrive/266_project/microaggressions_v1.json', lines=True)



In [ ]:
self_MA.head()

,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago
0,168667240562,quote,"""I'm probably such a racist, but a black man d...",(via microaggressions),None,None,http://www.microaggressions.com/post/168667240...,[],37,0,1 month ago
1,164870047110,chat,None,,[Teacher:: what do they look like? (referring ...,None,http://www.microaggressions.com/post/164870047...,"[ethnicity, religion]",38,0,5 months ago
2,164858087247,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,None,http://www.microaggressions.com/post/164858087...,"[race, ethnicity]",55,0,5 months ago
3,164834173567,text,None,I patiently waited after class to ask my profe...,None,None,http://www.microaggressions.com/post/164834173...,"[gender, survivor]",46,0,5 months ago
4,164821946327,quote,"""CAN YOU HEAR ME?""","The ""joking"" reaction of various friends, cowo...",None,None,http://www.microaggressions.com/post/164821946...,[ableism],65,0,5 months ago


In [ ]:
# Get all unique tags from the column
unique_tags = set(tag for tags in self_MA["tags"] if isinstance(tags, list) for tag in tags)

print(unique_tags)


{'genderqueer', 'submission', 'language', 'Survivor', 'nationality', 'eurocentrism', 'eurocentricism', 'reflection', 'mind', 'citizenship', 'sex', 'sexism', 'religion', 'ableism', 'Mind', 'resistance', 'ability', 'Skin Tone', 'race', 'region', 'invisibility', 'skin tone', 'patriarchy', 'event', 'body', 'education', 'Sexuality', 'acces', 'sexuality', 'neurodiversity', 'gender', 'ethnicity', 'age', 'colonialism', 'internalization', 'everything', 'immigration', 'discussion', 'invisible', 'faq', 'ody', 'marriage', 'immigration status', 'hair', 'trans', 'class', 'queer', 'meta', 'relationship', 'racism', '3yearsago', 'immigrant status', 'survivor', 'marriagestatus', 'institutional', 'reblog', 'school', 'family', 'mental health', 'macro', 'beauty', 'surivor', 'west', 'orientalism', 'trans*'}


In [ ]:
#filter to include only race-related tags

include = {
    "Skin Tone", "skin tone", "racism", "orientalism",
    "race", "ethnicity", "eurocentrism", "eurocentricism"
}

mask = self_MA["tags"].apply(lambda xs: any(t in include for t in xs))
self_MA_race = self_MA[mask].copy()


In [ ]:
#sanity check

self_MA_race.head()

,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago
1,164870047110,chat,None,,[Teacher:: what do they look like? (referring ...,None,http://www.microaggressions.com/post/164870047...,"[ethnicity, religion]",38,0,5 months ago
2,164858087247,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,None,http://www.microaggressions.com/post/164858087...,"[race, ethnicity]",55,0,5 months ago
5,164797618881,text,None,It angers me when people measure my race by th...,None,None,http://www.microaggressions.com/post/164797618...,"[race, gender]",45,0,5 months ago
6,164785208123,quote,"""All immigrants should go back to their own co...",My white American gay classmate to another fel...,None,None,http://www.microaggressions.com/post/164785208...,"[nationality, race, ethnicity, citizenship]",55,0,5 months ago
7,164760128313,quote,"""Whenever black people come to the beach I sta...","A prejudiced girl in front of me in line, on t...",None,None,http://www.microaggressions.com/post/164760128...,[race],47,0,5 months ago


In [ ]:
#sanity check, check to see that all rows have at least one of the race-related key words
unique_sets = (
    self_MA_race["tags"]
    .dropna()
    .loc[lambda s: s.apply(lambda x: isinstance(x, list))]
    .apply(lambda x: tuple(sorted(set(x))))   # canonicalize
    .unique()
)

print(unique_sets)

[('ethnicity', 'religion') ('ethnicity', 'race') ('gender', 'race')
 ('citizenship', 'ethnicity', 'nationality', 'race') ('race',)
 ('body', 'gender', 'race') ('gender', 'race', 'sexuality')
 ('language', 'race') ('class', 'race') ('ethnicity', 'nationality')
 ('internalization', 'race') ('class', 'gender', 'race')
 ('ethnicity', 'language') ('race', 'sexuality')
 ('language', 'nationality', 'race') ('ethnicity', 'race', 'religion')
 ('nationality', 'race', 'religion') ('body', 'race') ('race', 'religion')
 ('ethnicity', 'language', 'nationality', 'race') ('nationality', 'race')
 ('ethnicity', 'language', 'race') ('ethnicity', 'nationality', 'race')
 ('immigration', 'nationality', 'race') ('meta', 'nationality', 'race')
 ('class', 'ethnicity', 'race')
 ('immigration', 'immigration status', 'race') ('ethnicity',)
 ('meta', 'race') ('3yearsago', 'race', 'reblog') ('race', 'reblog')
 ('race', 'racism') ('gender', 'macro', 'race') ('macro', 'race')
 ('beauty', 'hair', 'race')
 ('class', 'e

In [ ]:
#check to make sure all entries are extracted from www.microaggressions.com

from urllib.parse import urlparse

# Trim whitespace first
perm = self_MA['permalink'].astype(str).str.strip()

# Strict pattern: only http/https and domain microaggressions.com (with or without www)
pattern = r'^https?://(www\.)?microaggressions\.com(/|$)'
mask_ok = perm.str.match(pattern, na=False)

# Bad rows = anything not matching (also include NaNs explicitly)
bad = self_MA.loc[~mask_ok | self_MA['permalink'].isna()].copy()

print("Bad row count:", bad.shape[0])
bad[['permalink']].head(20)  # preview



Bad row count: 0


,permalink


In [ ]:
#count the unique "types" of entries
self_MA_race["type"].value_counts()

,count
type,
text,615
quote,558
chat,215
photo,46
video,13
link,12


In [ ]:
#total number of values in data table
len(self_MA_race)

1459

In [ ]:
#what are "chat" columns?

chat_rows = self_MA[self_MA['type'] == 'chat']
chat_rows.head()


,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago
1,164870047110,chat,None,,[Teacher:: what do they look like? (referring ...,None,http://www.microaggressions.com/post/164870047...,"[ethnicity, religion]",38,0,5 months ago
12,164669364741,chat,None,,"[Old woman:: ""Schools shouldn't offer free lun...",None,http://www.microaggressions.com/post/164669364...,[class],98,1,5 months ago
14,164631768821,chat,None,,[At the office. Three Jamaican men with dreadl...,None,http://www.microaggressions.com/post/164631768...,"[race, gender]",25,0,5 months ago
28,164402283777,chat,None,,[Assistant:: I've never dated a white woman. I...,None,http://www.microaggressions.com/post/164402283...,[],124,0,5 months ago
97,153682668560,chat,None,,"[Me:: Hi, I'm here for college scholarship inf...",None,http://www.microaggressions.com/post/153682668...,[race],90,0,1 year ago


In [ ]:
link_rows = self_MA_race[self_MA_race['type'] == 'link']
link_rows

,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago
310,78777977867,link,None,,None,http://microaggressions.tumblr.com/tagged/xs/c...,http://www.microaggressions.com/post/787779778...,"[meta, race]",4,0,3 years ago
554,56599441708,link,None,,None,http://www.nytimes.com/2012/01/02/nyregion/sep...,http://www.microaggressions.com/post/565994417...,"[religion, ethnicity]",712,0,4 years ago
1308,14245739494,link,None,,None,http://www.forbes.com/sites/quickerbettertech/...,http://www.microaggressions.com/post/142457394...,"[race, class, meta]",126,0,6 years ago
1832,8914137754,link,None,,None,http://www.notracistbut.com/,http://www.microaggressions.com/post/891413775...,[race],24,0,6 years ago
1875,8696561560,link,None,,None,http://www.youtube.com/watch?feature=player_em...,http://www.microaggressions.com/post/869656156...,[race],76,0,6 years ago
1949,8308086409,link,None,,None,http://notracistbut.tumblr.com/,http://www.microaggressions.com/post/830808640...,"[race, meta, macro]",22,0,6 years ago
2102,7615740571,link,None,,None,http://www.slantedmagazine.com/speak-on-it-sub...,http://www.microaggressions.com/post/761574057...,"[race, meta]",24,0,6 years ago
2319,6490987433,link,None,,None,http://www.youtube.com/watch?v=pXG38QxXY-s,http://www.microaggressions.com/post/649098743...,"[race, gender, meta]",43,0,6 years ago
2580,5188797255,link,None,,None,http://indiancountrytodaymedianetwork.com/ict_...,http://www.microaggressions.com/post/518879725...,"[meta, race, macro]",75,0,6 years ago
2833,3913333950,link,None,,None,http://miswritten.tumblr.com/post/3893297537,http://www.microaggressions.com/post/391333395...,"[gender, race, meta]",246,0,6 years ago


In [ ]:
#keep only rows where labeled quote, or text in the "type" column
#at first I kept chat but then decided to take out since a transcript is a very different phenomenon

keep_types = {'text','quote'}
self_MA_race_filtertypes = self_MA_race[self_MA_race['type'].isin(keep_types)].copy()
self_MA_race_filtertypes.head()


,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago
2,164858087247,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,None,http://www.microaggressions.com/post/164858087...,"[race, ethnicity]",55,0,5 months ago
5,164797618881,text,None,It angers me when people measure my race by th...,None,None,http://www.microaggressions.com/post/164797618...,"[race, gender]",45,0,5 months ago
6,164785208123,quote,"""All immigrants should go back to their own co...",My white American gay classmate to another fel...,None,None,http://www.microaggressions.com/post/164785208...,"[nationality, race, ethnicity, citizenship]",55,0,5 months ago
7,164760128313,quote,"""Whenever black people come to the beach I sta...","A prejudiced girl in front of me in line, on t...",None,None,http://www.microaggressions.com/post/164760128...,[race],47,0,5 months ago
9,164721945947,quote,"""I think black girls with short hair are ugly.""",My ex after I told him that I was going to cut...,None,None,http://www.microaggressions.com/post/164721945...,"[race, gender, body]",28,0,5 months ago


In [ ]:
#total number of entries
len(self_MA_race_filtertypes)

1173

#Balance the Dataset with non-Microaggressions

I'm using Carlos' dataset to balance this one.

In [ ]:
import pandas as pd

balanced_micro = pd.read_csv('/content/drive/MyDrive/266_project/balanced_microaggressions.csv')

In [ ]:
balanced_micro.head()

,text,label
0,"“ So every week, do the Guatemalans come and m...",1
1,“ Why would you wear your hair like that today...,1
2,"“ You’re Asian, so you must be good at math, r...",1
3,"“ Oh, I don’t know how to pronounce those name...",1
4,“ You’re very exotic-looking. ”,1


In [ ]:
#count number of micro and non-microaggressions

balanced_micro["label"].value_counts()

,count
label,
1,585
0,585


In [ ]:
#add a label column to the selfMA dataset

self_MA_race_filtertypes["label"] = 1


In [ ]:
#sanity check
self_MA_race_filtertypes.head()

,id,type,quote,text,transcript,media_url,permalink,tags,n_hearts,n_comments,time_ago,label
2,164858087247,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,None,http://www.microaggressions.com/post/164858087...,"[race, ethnicity]",55,0,5 months ago,1
5,164797618881,text,None,It angers me when people measure my race by th...,None,None,http://www.microaggressions.com/post/164797618...,"[race, gender]",45,0,5 months ago,1
6,164785208123,quote,"""All immigrants should go back to their own co...",My white American gay classmate to another fel...,None,None,http://www.microaggressions.com/post/164785208...,"[nationality, race, ethnicity, citizenship]",55,0,5 months ago,1
7,164760128313,quote,"""Whenever black people come to the beach I sta...","A prejudiced girl in front of me in line, on t...",None,None,http://www.microaggressions.com/post/164760128...,[race],47,0,5 months ago,1
9,164721945947,quote,"""I think black girls with short hair are ugly.""",My ex after I told him that I was going to cut...,None,None,http://www.microaggressions.com/post/164721945...,"[race, gender, body]",28,0,5 months ago,1


In [ ]:
#further edit selfMA dataset so that only necessary columns are present, with the aim of eventually concatenating with balanced_micro

self_MA_race_FINAL = self_MA_race_filtertypes.drop(
    columns=["id", "media_url", "permalink", "n_hearts", "n_comments", "time_ago"]
)


In [ ]:
#sanity check

self_MA_race_FINAL.head()

,type,quote,text,transcript,tags,label
2,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,"[race, ethnicity]",1
5,text,None,It angers me when people measure my race by th...,None,"[race, gender]",1
6,quote,"""All immigrants should go back to their own co...",My white American gay classmate to another fel...,None,"[nationality, race, ethnicity, citizenship]",1
7,quote,"""Whenever black people come to the beach I sta...","A prejudiced girl in front of me in line, on t...",None,[race],1
9,quote,"""I think black girls with short hair are ugly.""",My ex after I told him that I was going to cut...,None,"[race, gender, body]",1


In [ ]:
#rename "text" column as "quote" in balanced_micro (since they're all quotes) with an eye to concatenating

balanced_micro.rename(
    columns={"text": "quote"},
    inplace=True
)


In [ ]:
balanced_micro.head()

,quote,label
0,"“ So every week, do the Guatemalans come and m...",1
1,“ Why would you wear your hair like that today...,1
2,"“ You’re Asian, so you must be good at math, r...",1
3,"“ Oh, I don’t know how to pronounce those name...",1
4,“ You’re very exotic-looking. ”,1


In [ ]:
#filter out rows with label "1"

balanced_micro_FINAL = balanced_micro[balanced_micro["label"] == 0]


In [ ]:
#sanity check
balanced_micro_FINAL["label"].value_counts()

,count
label,
0,585


In [ ]:
#add type column in balanced_micro dataset

balanced_micro_FINAL = balanced_micro_FINAL.assign(type="quote")



In [ ]:
balanced_micro_FINAL.head()

,quote,label,type
585,What another name for your black friends? Mudd...,0,quote
586,"""RT @_jadanoele: """"na boo you just be fucking ...",0,quote
587,Eating cold fries for lunch\nI am indeed a loser,0,quote
588,Kluna Eating Lego! Kluna Tik NVA Dinner #9| AS...,0,quote
589,Fingers crossed that the roads are still too f...,0,quote


In [ ]:
#combine datasets

combined_micro_dataset = pd.concat(
    [self_MA_race_FINAL, balanced_micro_FINAL],
    ignore_index=True,
    sort=False
)

In [ ]:
combined_micro_dataset.head()

,type,quote,text,transcript,tags,label
0,quote,"""Yeah, but you're not that kind of Native.""",This has happened more than once. A person co...,None,"[race, ethnicity]",1
1,text,None,It angers me when people measure my race by th...,None,"[race, gender]",1
2,quote,"""All immigrants should go back to their own co...",My white American gay classmate to another fel...,None,"[nationality, race, ethnicity, citizenship]",1
3,quote,"""Whenever black people come to the beach I sta...","A prejudiced girl in front of me in line, on t...",None,[race],1
4,quote,"""I think black girls with short hair are ugly.""",My ex after I told him that I was going to cut...,None,"[race, gender, body]",1


In [ ]:
combined_micro_dataset["label"].value_counts()

,count
label,
1,1173
0,585


Dataset is not yet completely balanced. Maybe we should add more 0's.

In [ ]:
#get ready for training

df_micro = combined_micro_dataset.copy()




In [ ]:
#imports
!pip -q install transformers accelerate datasets evaluate scikit-learn
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict



import transformers
import evaluate

!pip install -q torchinfo
from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
#split dataset

train_df, tmp_df = train_test_split(df_micro, test_size=0.2, random_state=42, stratify=df_micro['label'])
valid_df, test_df = train_test_split(tmp_df, test_size=0.5, random_state=42, stratify=tmp_df['label'])
ds = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(valid_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True)),
})


In [ ]:
!pip install -U transformers




model_checkpoint_name = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_name)
bert_classification_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint_name,
    num_labels=2)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#tokenize text

max_sequence_length = 128

from typing import List
from itertools import zip_longest

# QUOTE-ONLY
def preprocess_quote_only(batch):
    # make sure we pass a list[str]
    quotes: List[str] = [
        q if isinstance(q, str) else ""
        for q in batch["quote"]
    ]
    enc = tokenizer(
        quotes,
        max_length=max_sequence_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
    )
    enc["labels"] = batch["label"]
    return enc

# QUOTE + TEXT (also handles text-only and quote-only rows)
def preprocess_quote_plus_text(batch):
    quotes = [q if isinstance(q, str) else "" for q in batch.get("quote", [])]
    texts  = [t if isinstance(t, str) else "" for t in batch.get("text",  [])]

    # pack: "quote [SEP] text" if both; otherwise whichever exists
    packed = []
    for q, t in zip_longest(quotes, texts, fillvalue=""):
        if q and t:
            packed.append(f"{q} [SEP] {t}")
        elif q:
            packed.append(q)
        else:
            packed.append(t)

    enc = tokenizer(
        packed,
        max_length=max_sequence_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
    )
    enc["labels"] = batch["label"]
    return enc



In [ ]:
#preprocess data
from datasets import Dataset

# 1) Convert pandas → Dataset
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

# 2) Build removal list (we only keep the tokenized columns + labels from the map output)
cols_to_drop_train = train_ds.column_names
cols_to_drop_valid = valid_ds.column_names

# 3) Preprocess: QUOTE-ONLY
preprocessed_train_data_quote = train_ds.map(
    preprocess_quote_only,
    batched=True,
    remove_columns=cols_to_drop_train
)
preprocessed_dev_data_quote = valid_ds.map(
    preprocess_quote_only,
    batched=True,
    remove_columns=cols_to_drop_valid
)

# 4) Preprocess: QUOTE + TEXT (text-only rows are included by your function)
preprocessed_train_data_quotetext = train_ds.map(
    preprocess_quote_plus_text,
    batched=True,
    remove_columns=cols_to_drop_train
)
preprocessed_dev_data_quotetext = valid_ds.map(
    preprocess_quote_plus_text,
    batched=True,
    remove_columns=cols_to_drop_valid
)

# 5) (Optional) Set format for PyTorch
cols = ["input_ids", "attention_mask", "token_type_ids", "labels"]
preprocessed_train_data_quote.set_format(type="torch", columns=cols)
preprocessed_dev_data_quote.set_format(type="torch", columns=cols)
preprocessed_train_data_quotetext.set_format(type="torch", columns=cols)
preprocessed_dev_data_quotetext.set_format(type="torch", columns=cols)


Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

In [ ]:
#create evaluation metric
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

def fine_tune_classification_model(classification_model,
                                   train_dataset,
                                   dev_dataset,
                                   batch_size=16,
                                   num_epochs=3,
                                   learning_rate=2e-5,
                                   output_dir="FT_micro"):
    """
    Uses pre-tokenized datasets (with input_ids, attention_mask, [token_type_ids], labels).
    Trains and evaluates once per epoch. Compatible with older HF versions that use `eval_strategy`.
    """

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        report_to="none",
        fp16=True                   # set False if your env/GPU doesn't support FP16
    )

    trainer = Trainer(
        model=classification_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    return trainer


In [ ]:
# Experiment 1: QUOTE-ONLY
trainer_quote = fine_tune_classification_model(
    classification_model=bert_classification_model,
    train_dataset=preprocessed_train_data_quote,
    dev_dataset=preprocessed_dev_data_quote,
    batch_size=16,
    num_epochs=3,
    learning_rate=2e-5,
    output_dir="FT_quote_only"
)

# Experiment 2: QUOTE + TEXT (context; also handles text-only rows)
trainer_quotetext = fine_tune_classification_model(
    classification_model=bert_classification_model,
    train_dataset=preprocessed_train_data_quotetext,
    dev_dataset=preprocessed_dev_data_quotetext,
    batch_size=16,
    num_epochs=3,
    learning_rate=2e-5,
    output_dir="FT_quote_plus_text"
)


W1107 23:24:18.864000 323 torch/_inductor/utils.py:1436] [1/0_1] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss,Accuracy
1,0.074000,0.111802,0.988636
2,0.013500,0.119892,0.988636
3,0.000000,0.108302,0.988636


Epoch,Training Loss,Validation Loss,Accuracy
1,0.185900,0.180370,0.977273
2,0.013500,0.166199,0.982955
3,0.000800,0.171155,0.982955
